# 汇聚层
:label:`sec_pooling`

通常当我们处理图像时，我们希望逐渐降低隐藏表示的空间分辨率、聚集信息，这样随着我们在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大。

而我们的机器学习任务通常会跟全局图像的问题有关（例如，“图像是否包含一只猫呢？”），所以我们最后一层的神经元应该对整个输入的全局敏感。通过逐渐聚合信息，生成越来越粗糙的映射，最终实现学习全局表示的目标，同时将卷积图层的所有优势保留在中间层。


Note：**将卷积图层的所有优势保留在中间层**
这是最关键的一点！它强调：
- 中间层仍保持高分辨率 + 局部特征提取能力
- 卷积层的优势包括：
    - 局部连接（local connectivity）
    - 权值共享（parameter sharing）
    - 平移等变性（translation equivariance）
    - 层次化特征学习

✅ 池化没有取代卷积，而是在卷积提取的丰富中间特征基础上做摘要 → 低层保留细节（用于定位、分割），高层提供语义（用于分类）

这句话的本质是：
>CNN 通过“卷积提取 + 池化摘要”的层级流水线，在保留局部特征表达能力的同时，逐步构建出对输入的全局语义理解。

这是一种多尺度、层次化的表示学习策略，也是 CNN 成功的关键之一。



此外，当检测较底层的特征时（例如 :numref:`sec_conv_layer`中所讨论的边缘），我们通常希望这些特征*保持某种程度上的平移不变性*。例如，如果我们拍摄黑白之间轮廓清晰的图像`X`，并将整个图像向右移动一个像素，即`Z[i, j] = X[i, j + 1]`，则新图像`Z`的输出可能大不相同。而在现实中，随着拍摄角度的移动，任何物体几乎不可能发生在同一像素上。即使用三脚架拍摄一个静止的物体，由于快门的移动而引起的相机振动，可能会使所有物体左右移动一个像素（除了高端相机配备了特殊功能来解决这个问题）。

本节将介绍*汇聚*（pooling）层，它具有双重目的：降低卷积层对位置的敏感性，同时降低对空间降采样表示的敏感性。

## 最大汇聚层和平均汇聚层

与卷积层类似，汇聚层运算符由一个固定形状的窗口组成，该窗口根据其步幅大小在输入的所有区域上滑动，为固定形状窗口（有时称为*汇聚窗口*）遍历的每个位置计算一个输出。
然而，不同于卷积层中的输入与卷积核之间的互相关计算，汇聚层不包含参数。
相反，池运算是确定性的，我们通常计算汇聚窗口中所有元素的最大值或平均值。这些操作分别称为*最大汇聚层*（maximum pooling）和*平均汇聚层*（average pooling）。

在这两种情况下，与互相关运算符一样，汇聚窗口从输入张量的左上角开始，从左往右、从上往下的在输入张量内滑动。在汇聚窗口到达的每个位置，它计算该窗口中输入子张量的最大值或平均值。计算最大值或平均值是取决于使用了最大汇聚层还是平均汇聚层。

![汇聚窗口形状为 $2\times 2$ 的最大汇聚层。着色部分是第一个输出元素，以及用于计算这个输出的输入元素: $\max(0, 1, 3, 4)=4$.](../img/pooling.svg)
:label:`fig_pooling`

 :numref:`fig_pooling`中的输出张量的高度为$2$，宽度为$2$。这四个元素为每个汇聚窗口中的最大值：

$$
\max(0, 1, 3, 4)=4,\\
\max(1, 2, 4, 5)=5,\\
\max(3, 4, 6, 7)=7,\\
\max(4, 5, 7, 8)=8.\\
$$

汇聚窗口形状为$p \times q$的汇聚层称为$p \times q$汇聚层，汇聚操作称为$p \times q$汇聚。

回到本节开头提到的对象边缘检测示例，现在我们将使用卷积层的输出作为$2\times 2$最大汇聚的输入。
设置卷积层输入为`X`，汇聚层输出为`Y`。
无论`X[i, j]`和`X[i, j + 1]`的值相同与否，或`X[i, j + 1]`和`X[i, j + 2]`的值相同与否，汇聚层始终输出`Y[i, j] = 1`。
也就是说，使用$2\times 2$最大汇聚层，即使在高度或宽度上移动一个元素，卷积层仍然可以识别到模式。

在下面的代码中的`pool2d`函数，我们(**实现汇聚层的前向传播**)。
这类似于 :numref:`sec_conv_layer`中的`corr2d`函数。
然而，这里我们没有卷积核，输出为输入中每个区域的最大值或平均值。


In [1]:
import torch
from torch import nn
from myd2l import torch as d2l

In [2]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

我们可以构建 :numref:`fig_pooling`中的输入张量`X`，[**验证二维最大汇聚层的输出**]。


In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

此外，我们还可以(**验证平均汇聚层**)。


In [4]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

## [**填充和步幅**]

与卷积层一样，汇聚层也可以改变输出形状。和以前一样，我们可以通过填充和步幅以获得所需的输出形状。
下面，我们用深度学习框架中内置的二维最大汇聚层，来演示汇聚层中填充和步幅的使用。
我们首先构造了一个输入张量`X`，它有四个维度，其中样本数和通道数都是1。


In [5]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

默认情况下，(**深度学习框架中的步幅与汇聚窗口的大小相同**)。
因此，如果我们使用形状为`(3, 3)`的汇聚窗口，那么默认情况下，我们得到的步幅形状为`(3, 3)`。


In [6]:
pool2d = nn.MaxPool2d(3)
# kernel_size = 3
# 默认 stride = kernel_size = 3（因为未指定）
# 默认 padding = 0
pool2d(X)

tensor([[[[10.]]]])

🔢 已知条件

```python
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
# X.shape = (N, C, H, W) = (1, 1, 4, 4)

pool2d = nn.MaxPool2d(3)
# kernel_size = 3
# 默认 stride = kernel_size = 3（因为未指定）
# 默认 padding = 0
```

> ✅ **PyTorch 中 `nn.MaxPool2d(kernel_size)` 的默认行为**：
> - `stride = kernel_size`
> - `padding = 0`
> - `dilation = 1`
> - `ceil_mode = False`

---

📐 输出尺寸计算公式（MaxPool / AvgPool）

对于高度维度（宽度同理）：

$$
H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} + 2 \times \text{padding} - \text{dilation} \times (k_h - 1) - 1}{\text{stride}} + 1 \right\rfloor
$$

在标准池化中（`dilation=1`），简化为：

$$
H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} + 2p - k}{s} \right\rfloor + 1
$$

---

🧮 代入你的数值

- $ H_{\text{in}} = 4 $
- $ k = 3 $
- $ p = 0 $
- $ s = 3 $（因为 `stride` 默认等于 `kernel_size`）

$$
H_{\text{out}} = \left\lfloor \frac{4 + 0 - 3}{3} \right\rfloor + 1 = \left\lfloor \frac{1}{3} \right\rfloor + 1 = 0 + 1 = 1
$$

同理，$ W_{\text{out}} = 1 $



[**填充和步幅可以手动设定**]。


In [7]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

当然，我们可以(**设定一个任意大小的矩形汇聚窗口，并分别设定填充和步幅的高度和宽度**)。


In [8]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

## 多个通道

在处理多通道输入数据时，[**汇聚层在每个输入通道上单独运算**]，而不是像卷积层一样在通道上对输入进行汇总。
这意味着汇聚层的输出通道数与输入通道数相同。
下面，我们将在通道维度上连结张量`X`和`X + 1`，以构建具有2个通道的输入。


In [9]:
X = torch.cat((X, X + 1), dim=1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

如下所示，汇聚后输出通道的数量仍然是2。


In [10]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

## 小结

* 对于给定输入元素，最大汇聚层会输出该窗口内的最大值，平均汇聚层会输出该窗口内的平均值。
* 汇聚层的主要优点之一是减轻卷积层对位置的过度敏感。
* 我们可以指定汇聚层的填充和步幅。
* 使用最大汇聚层以及大于1的步幅，可减少空间维度（如高度和宽度）。
* 汇聚层的输出通道数与输入通道数相同。


池化层（Pooling Layer）是卷积神经网络（CNN）中的核心组件之一，主要用于**降低特征图的空间维度、增强平移不变性、减少计算量和过拟合风险**。下面我们系统地回答你的问题。

---

## ✅ 一、什么是池化层？

池化层是一种**固定的、无参数的下采样操作**，它在输入特征图的局部区域（称为“池化窗口”）上应用一个**聚合函数**（如最大值、平均值），输出该区域的代表值。

> 📌 **关键特点**：  
> - **没有可学习参数**（即没有权重和偏置）  
> - **不是线性变换**，而是**非线性下采样**  
> - 操作是**确定性的、手工设计的**

---

## ✅ 二、池化层的作用与意义

| 作用 | 说明 |
|------|------|
| **1. 降维（Downsampling）** | 减少特征图的高和宽，降低后续层的计算量和内存占用 |
| **2. 平移不变性（Translation Invariance）** | 即使目标在图像中轻微移动，池化后的响应变化不大（尤其 Max Pooling） |
| **3. 防止过拟合** | 通过丢弃部分空间信息，减少模型对细节的依赖 |
| **4. 扩大感受野** | 后续层能“看到”更大的原始图像区域 |
| **5. 保留显著特征** | 如 Max Pooling 保留最激活的特征，适合检测纹理、边缘等 |

> 💡 **类比**：  
> 如果卷积层是“显微镜”（提取局部特征），池化层就是“摘要器”（提炼关键信息）。

---

## ✅ 三、池化层 vs 卷积层

| 特性 | 池化层 | 卷积层 |
|------|--------|--------|
| **是否有可学习参数** | ❌ 无 | ✅ 有权重（和偏置） |
| **是否需要训练** | ❌ 固定操作 | ✅ 需要反向传播更新 |
| **操作类型** | 聚合（max/avg） | 线性滤波 + 非线性激活 |
| **目的** | 下采样 + 不变性 | 特征提取 |
| **感受野影响** | 扩大后续感受野 | 构建局部感受野 |
| **输出通道数** | = 输入通道数 | 可自定义（由卷积核数量决定） |

> 🔍 **重要区别**：  
> 卷积层**学习**如何提取特征，池化层**固定规则**地压缩特征。

---

## ✅ 四、池化层的主要类型

### 1. **最大池化（Max Pooling）**
- **操作**：取池化窗口内的**最大值**
- **优点**：保留最显著的激活，对噪声鲁棒，增强平移不变性
- **常用场景**：几乎所有 CNN（如 AlexNet, ResNet）

### 2. **平均池化（Average Pooling）**
- **操作**：取池化窗口内的**平均值**
- **优点**：平滑特征，保留背景信息
- **常用场景**：全局池化（Global Average Pooling）、某些轻量模型

### 3. **其他类型（较少用）**
- **L2 池化**：取窗口内 L2 范数
- **随机池化（Stochastic Pooling）**：按激活值比例随机采样（用于正则化）
- **自适应池化（Adaptive Pooling）**：自动计算 kernel/stride 以得到指定输出尺寸

---

## ✅ 五、超参数（Hyperparameters）与可学习参数

### 📌 所有池化层的共同点：
- **没有可学习参数**（`weight` 和 `bias` 都不存在）
- 所有行为由**超参数**控制

### 各类型池化层的超参数：

| 超参数 | 含义 | 是否必需 |
|--------|------|--------|
| **`kernel_size`** | 池化窗口大小（如 `(2,2)`） | ✅ 是 |
| **`stride`** | 步幅（默认 = `kernel_size`） | ⚠️ 可选（通常省略） |
| **`padding`** | 填充（较少用，因池化不需补零） | ❌ 通常不用 |
| **`ceil_mode`**（PyTorch） | 是否向上取整输出尺寸 | 可选 |

#### 示例（PyTorch）：
```python
# 最大池化
nn.MaxPool2d(kernel_size=2, stride=2)   # 输出尺寸减半

# 平均池化
nn.AvgPool2d(kernel_size=3, stride=1, padding=1)

# 全局平均池化（输出 1x1）
nn.AdaptiveAvgPool2d((1, 1))
```

> ✅ **总结**：  
> - **超参数**：`kernel_size`, `stride`, `padding`（极少用）  
> - **可学习参数**：**0 个**（池化层不可训练）

---

## ✅ 六、特殊形式：全局池化（Global Pooling）

- **全局最大池化（Global Max Pooling）**：`kernel_size = (H, W)` → 输出 `(C, 1, 1)`
- **全局平均池化（Global Average Pooling, GAP）**：  
  - 广泛用于分类网络末端（替代全连接层）
  - **优势**：减少参数、防止过拟合、提供可解释性（CAM 可视化）

```python
# 替代全连接层的经典用法
net = nn.Sequential(
    ...,                    # 特征提取
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(c, num_classes)  # 只需 c × num_classes 参数
)
```

---

## ✅ 七、池化层的争议与现代趋势

- **争议**：池化会**丢失精确空间信息**，不利于像素级任务（如语义分割）
- **现代替代方案**：
  - 使用 **stride > 1 的卷积** 代替池化（如 ResNet）
  - 使用 **可学习下采样**（如反卷积、注意力下采样）
- **但池化仍在广泛使用**：因其简单、高效、有效

---

## ✅ 总结表

| 项目 | 内容 |
|------|------|
| **定义** | 对局部区域进行聚合的下采样操作 |
| **作用** | 降维、平移不变性、防过拟合、扩大感受野 |
| **vs 卷积** | 无参数 vs 有参数；固定规则 vs 可学习 |
| **主要类型** | Max Pooling, Avg Pooling, Adaptive Pooling |
| **超参数** | `kernel_size`, `stride`, （极少用 `padding`） |
| **可学习参数** | **0 个** |
| **典型应用** | CNN 中间层降维、GAP 用于分类头 |

> 💡 **记住**：  
> **池化层是“免费的”下采样工具——它不增加模型参数，却带来强大的归纳偏置。**


## 练习

1. 尝试将平均汇聚层作为卷积层的特殊情况实现。
1. 尝试将最大汇聚层作为卷积层的特殊情况实现。
1. 假设汇聚层的输入大小为$c\times h\times w$，则汇聚窗口的形状为$p_h\times p_w$，填充为$(p_h, p_w)$，步幅为$(s_h, s_w)$。这个汇聚层的计算成本是多少？
1. 为什么最大汇聚层和平均汇聚层的工作方式不同？
1. 我们是否需要最小汇聚层？可以用已知函数替换它吗？
1. 除了平均汇聚层和最大汇聚层，是否有其它函数可以考虑（提示：回想一下`softmax`）？为什么它不流行？


[Discussions](https://discuss.d2l.ai/t/1857)
